# Single merge script

This script runs a single layer join on hex grid. Uses Kate's generated hexes and Inputs.<br>
There is no cleaning and type controll. No optimization/vectorization/parallelization. 

In [1]:
# Import modules
import geopandas as gpd
import fiona

Modify below cell to indicate path to database, hex level and joined layer

In [2]:
# Define path to hex grid database
hex_gdb = r'C://Research/Grid_effort/H3_5_10_Grd.gdb'
# Define hex level
hexLevel = '5'

# Define the path to the geodatabase with joined layer
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
# Name of layer to join
layer_to_join = 'tj_2021_us_st_cnt'

Run below cell to perform merge and save result

In [3]:
# Read in the hex polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
# Read the joined layer
joined_layer = gpd.read_file(inputs_gdb, layer=layer_to_join)
# Check if garbage data is present and try to remove it
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')
# Compare the CRS and use hex coordinates, as it is in meters. Projecting to WGS84 lowers performance 
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)
# Perform spatial join
# here a modification migh be needed, to include different predicates parameter, to differ results
# more here: https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates
output_layer = gpd.sjoin(base_layer, joined_layer, how="left")
# Save the result in work path
output_layer.to_file('h'+hexLevel+'_'+layer_to_join+'.gdb',driver='OpenFileGDB')